In [24]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
import pyspark.sql as sql
from pyspark.sql.functions import col
from datetime import datetime
from typing import NamedTuple
import pyspark.sql.functions as f
import os

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'
conf = SparkConf().setAppName("Lab2").setMaster('yarn')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [3]:
languages_data = spark.read.csv("programming-languages.csv")

In [4]:
languages = languages_data.rdd.map(lambda x: x[0]) \
                  .map(lambda x: x.lower()) \
                  .map(lambda x: x.split(' ')[0]) \
                  .distinct() \
                  .collect()[1:]
languages[1:10]

['a-0', 'a+', 'a++', 'abap', 'abc', 'abset', 'absys', 'acc', 'accent']

In [5]:


posts_data =  spark.read.format("xml") \
                        .option("rootTag", "posts") \
                        .option("rowTag", "row") \
                        .load("posts_sample.xml")
posts_data

DataFrame[_AcceptedAnswerId: bigint, _AnswerCount: bigint, _Body: string, _ClosedDate: timestamp, _CommentCount: bigint, _CommunityOwnedDate: timestamp, _CreationDate: timestamp, _FavoriteCount: bigint, _Id: bigint, _LastActivityDate: timestamp, _LastEditDate: timestamp, _LastEditorDisplayName: string, _LastEditorUserId: bigint, _OwnerDisplayName: string, _OwnerUserId: bigint, _ParentId: bigint, _PostTypeId: bigint, _Score: bigint, _Tags: string, _Title: string, _ViewCount: bigint, _corrupt_record: string]

In [16]:

def initPost(posts):
    class Post(NamedTuple):
        view_count: float
        language_id: int
        year: int
        
    for post in posts:
        try:
            yield Post(
                view_count = float(post._ViewCount), 
                language_id = languages.index(post._Tags[(post._Tags.find('<')+1) : post._Tags.find('>')]),
                year = post._CreationDate.year
            )
        except:
            pass

In [19]:
all_years = posts_data.rdd \
      .filter(lambda x: x._Tags is not None) \
      .map(lambda post: post._CreationDate.year) \
      .distinct() \
      .sortBy(lambda x: x, ascending=True) \
      .collect() 
all_years

[2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

In [34]:

top_by_year = {}
for year in all_years:
    top_by_year[year] = posts_data.rdd \
              .filter(lambda x: x._Tags is not None) \
              .mapPartitions(initPost) \
              .filter(lambda x: x.year == year) \
              .map(lambda x: (x.language_id, x.view_count)) \
              .reduceByKey(lambda x, y: x + y) \
              .map(lambda x: (languages[x[0]], int(x[1]))) \
              .sortBy(lambda x: x[1], ascending=False) \
              .toDF() \
              .select(col("_1").alias("Language"), col("_2").alias(f"Views")) \
              .limit(10)

In [36]:
for year in all_years:
    print(f'TOP {year}')
    top_by_year[year].show()
    top_by_year[year].write.format("parquet").save(f"top_10_programming_languages_{year}")

TOP 2008
+--------+------+
|Language| Views|
+--------+------+
|      c#|790973|
+--------+------+

TOP 2009
+--------+------+
|Language| Views|
+--------+------+
|      c#|406966|
|  python|141564|
|     c++| 84914|
|    java| 67051|
|     php| 53449|
| windows|  9222|
|      f#|  5107|
| haskell|  4745|
|       c|  2467|
|  delphi|  2128|
+--------+------+

TOP 2010
+-----------+-------+
|   Language|  Views|
+-----------+-------+
|        php|1179230|
|       java| 506358|
| javascript| 234900|
|         c#| 213021|
|objective-c|  62563|
|          c|  55089|
|        c++|  50411|
|     python|  45184|
|       ruby|  16928|
|    windows|  15152|
+-----------+-------+

TOP 2011
+-----------+------+
|   Language| Views|
+-----------+------+
| javascript|587109|
|         c#|260339|
|       java|155049|
|     python|117481|
|        c++| 99426|
|          c| 61929|
|        php| 53623|
|objective-c| 23602|
|       bash| 16091|
|          r| 12764|
+-----------+------+

TOP 2012
+------